In [2]:
!pip install Whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=a9ec63674d055f3efd0d7c15c057e6a6eef80a507fe0a66f296bbd0c2572d9f4
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built Whisper


In [3]:
!pip install pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00


In [4]:
pip install -U yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.1 MB/s eta 0:00:00


In [5]:
pip install -U openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s

In [23]:
import os
import yt_dlp
import whisper
import nltk
from pytube import YouTube
from nltk.tokenize import sent_tokenize
import requests
import time
from textblob import TextBlob

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [20]:


def get_subtitles(video_url):
    """Extracts subtitles using yt-dlp if available."""
    ydl_opts = {
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['en'],
        'skip_download': True,
        'outtmpl': '%(id)s.%(ext)s'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(video_url, download=False)
            video_id = info.get('id')
        except yt_dlp.utils.DownloadError as e:
            if "Video unavailable" in str(e):
                print(f"Error: Video with URL '{video_url}' is geo-restricted or unavailable in your region.")
                return None
            else:
                raise

    subtitle_file = f"{video_id}.en.vtt"
    return subtitle_file if os.path.exists(subtitle_file) else None

def download_audio(video_url):
    """Downloads audio using yt-dlp."""
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'audio.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    return "audio.mp3"

def transcribe_audio(audio_path):
    """Converts speech to text using Whisper model."""
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result['text']

def summarize_text(text, num_sentences=3):
    """Summarizes extracted text by selecting key sentences."""
    sentences = sent_tokenize(text)
    return " ".join(sentences[:num_sentences])

def analyze_sentiment(text):
    """Performs sentiment analysis using TextBlob."""
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity

    if sentiment_score > 0.05:
        return "Positive"
    elif sentiment_score < -0.05:
        return "Negative"
    else:
        return "Neutral"

def process_video(video_url):
    """Main function to extract, summarize, and analyze sentiment from a YouTube video."""
    subtitle_file = get_subtitles(video_url)

    if subtitle_file:
        with open(subtitle_file, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        audio_file = download_audio(video_url)
        text = transcribe_audio(audio_file)
        os.remove(audio_file)  # Cleanup

    summary = summarize_text(text)
    sentiment = analyze_sentiment(summary)

    return summary, sentiment

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [22]:
video_urls = [
    "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
    "https://www.youtube.com/watch?v=3JZ_D3ELwOQ",
    "https://www.youtube.com/watch?v=Zi_XLOBDo_Y",
    "https://www.youtube.com/watch?v=kJQP7kiw5Fk",
    "https://www.youtube.com/watch?v=9bZkp7q19f0"
]

for idx, video_url in enumerate(video_urls, 1):
    summary, sentiment = process_video(video_url)
    print(f"**Video {idx}:**")
    print(f"- **Summary:** \"{summary}\"")
    print(f"- **Sentiment:** {sentiment}\n")


[youtube] Extracting URL: https://www.youtube.com/watch?v=dQw4w9WgXcQ
[youtube] dQw4w9WgXcQ: Downloading webpage
[youtube] dQw4w9WgXcQ: Downloading tv client config
[youtube] dQw4w9WgXcQ: Downloading player f6e09c70
[youtube] dQw4w9WgXcQ: Downloading tv player API JSON
[youtube] dQw4w9WgXcQ: Downloading ios player API JSON
[youtube] dQw4w9WgXcQ: Downloading m3u8 information
[info] dQw4w9WgXcQ: Downloading subtitles: en
[youtube] Extracting URL: https://www.youtube.com/watch?v=dQw4w9WgXcQ
[youtube] dQw4w9WgXcQ: Downloading webpage
[youtube] dQw4w9WgXcQ: Downloading tv client config
[youtube] dQw4w9WgXcQ: Downloading player f6e09c70
[youtube] dQw4w9WgXcQ: Downloading tv player API JSON
[youtube] dQw4w9WgXcQ: Downloading ios player API JSON
[youtube] dQw4w9WgXcQ: Downloading m3u8 information
[info] dQw4w9WgXcQ: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of    3.28MiB in 00:00:00 at 26.01MiB/s  
[ExtractAudio] Destination: audio.mp3
Deleting original fi

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


**Video 1:**
- **Summary:** " Music There are no strangers to love You know the rules and something I've no commitments for the weekend of You wouldn't get this wrong any other guy I just wanna tell you how I feel I gotta make you understand Never gonna give you up Never gonna let you down Never gonna run around it He's hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell him how I And hurt you We've known each other for so long Your heart's been aching but You're too shy to say it It's as if we've all known what's been going on We know the game I'm with Gonna play it And if you ask me how I feel I gotta make you understand Never gonna give you up Never gonna let you down Never gonna run around it He's hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell him how I And hurt you Never gonna give you up Never gonna let you down Never gonna run around it He's hurt you Never gonna make you cry Never gonna say goodbye never gonna tell him how I And

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


**Video 2:**
- **Summary:** " Do you guys go to the gym? I go to the gym. But you already knew that."
- **Sentiment:** Neutral

[youtube] Extracting URL: https://www.youtube.com/watch?v=Zi_XLOBDo_Y
[youtube] Zi_XLOBDo_Y: Downloading webpage
[youtube] Zi_XLOBDo_Y: Downloading tv client config
[youtube] Zi_XLOBDo_Y: Downloading player f6e09c70
[youtube] Zi_XLOBDo_Y: Downloading tv player API JSON
[youtube] Zi_XLOBDo_Y: Downloading ios player API JSON
[youtube] Zi_XLOBDo_Y: Downloading m3u8 information
[info] Zi_XLOBDo_Y: Downloading subtitles: en
[youtube] Extracting URL: https://www.youtube.com/watch?v=Zi_XLOBDo_Y
[youtube] Zi_XLOBDo_Y: Downloading webpage
[youtube] Zi_XLOBDo_Y: Downloading tv client config
[youtube] Zi_XLOBDo_Y: Downloading player f6e09c70
[youtube] Zi_XLOBDo_Y: Downloading tv player API JSON
[youtube] Zi_XLOBDo_Y: Downloading ios player API JSON
[youtube] Zi_XLOBDo_Y: Downloading m3u8 information
[info] Zi_XLOBDo_Y: Downloading 1 format(s): 251
[download] Destination:

...

**Video 3:**
- **Summary:** " She was more like a beauty queen from a movie scene I shared her mind but don't do you mean I am the one Y'all good dance on the floor and around She said I am the one, good dance on the floor and around She told me her name is Billie Jean I said I am the one, good dance on the floor and around She told me her name is Billie Jean I shared her mind but don't do you mean I am the one Y'all good dance on the floor and around She told me her name is Billie Jean Where do I go ina wonder what you're doing I do but don't do you mean I am the one who you're not I do but don't do you mean I am the one I do but don't do you mean I am the one Turn that eye on the world, Turn that lower circle back, Turn that lower circle back Turn the lower circle back to frame For four to four to nine I was on the side But who can stand when he's in the manners? He's in the lands I'm going dance for the blow in round Something might throw advice Just remember to always think twice D

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


**Video 4:**
- **Summary:** " Eighth F particular character, Baby, get going super fast Fancy! Oh, oh no, oh no, oh no, oh, oh oh, oh-oh. Yo, si, sabes que ya llevo un rato mirándote."
- **Sentiment:** Positive

[youtube] Extracting URL: https://www.youtube.com/watch?v=9bZkp7q19f0
[youtube] 9bZkp7q19f0: Downloading webpage
[youtube] 9bZkp7q19f0: Downloading tv client config
[youtube] 9bZkp7q19f0: Downloading player f6e09c70
[youtube] 9bZkp7q19f0: Downloading tv player API JSON
[youtube] 9bZkp7q19f0: Downloading ios player API JSON
[youtube] 9bZkp7q19f0: Downloading m3u8 information
[info] 9bZkp7q19f0: Downloading subtitles: en
[youtube] Extracting URL: https://www.youtube.com/watch?v=9bZkp7q19f0
[youtube] 9bZkp7q19f0: Downloading webpage
[youtube] 9bZkp7q19f0: Downloading tv client config
[youtube] 9bZkp7q19f0: Downloading player f6e09c70
[youtube] 9bZkp7q19f0: Downloading tv player API JSON
[youtube] 9bZkp7q19f0: Downloading ios player API JSON
[youtube] 9bZkp7q19f0: Downloading m3u8 

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


**Video 5:**
- **Summary:** " BGM 호빤방담스던데 나 재는 따사로 온 인간적인 여자 커피한 잔에 여유로 하는 품격 있는 여자 밤이 어며 심장이 뜨거워지는 여자 그런 반 전 있는 여자 너는 선호해 나제는 너만큼 다사로운 그런 선호해 곧 비식이 도전해 원샷 때리는 선호해 밤이 오면 심장이 터져버린 선호해 그런 선호해 아름다워 사랑스러워 그래 너 해 그래 바로 너 해 아름다워 사랑스러워 그래 너 해 그래 바로 너 해 지금부터 그때까지 가볼까 그래 너 해 오빤 강남스타 강남스타 오빤 강남스타 강남스타 오빤 강남스타 에이 섹스 레이레 오빤 강남스타 에이 섹스 레이레 오빤 강남스타 청숙해 보이지만 볼 땐 너는 여자 이때다 싶으면 묶었던 머리 푸는 여자 가렸지만 왠 만한 노출 보다 야한 여자 그런 감각적인 여자 넌은 선호해 점점아 보이지만 볼 땐 너는 선호해 내가 되면 완전 미쳐 버리는 선호해 그녀 보다 사상이 올통벌통한 선호해 그런 선호해 아름다워 사랑스러워 그래 너 해 그래 바로 너 해 아름다워 사랑스러워 그래 너 해 그래 바로 너 해 지금부터 갈 때까지 가볼까 오빤 강남스타 강남스타 오빤 강남스타 강남스타 오빤 강남스타 에이 섹스 레이레 오빤 강남스타 에이 섹스 레이레 오빤 강남스타 비눈놈 그 위에 난은 넌 Baby Baby 난은 넌 좀 안은 넌 비눈놈 그 위에 난은 넌 Baby Baby 난은 넌 좀 안은 넌 좀 안은 넌 오빤 강남스타 에이 섹스 레이레 오빤 강남스타 에이 섹스 레이레 오빤 강남스타 오빤 강남스타"
- **Sentiment:** Neutral

